In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

In [ ]:
filenames = ["data/msn_02_fuel_leak_signals_preprocessed.csv", "data/msn_10_fuel_leak_signals_preprocessed.csv", "data/msn_11_fuel_leak_signals_preprocessed.csv", "data/msn_12_fuel_leak_signals_preprocessed.csv", "data/msn_14_fuel_leak_signals_preprocessed.csv", "data/msn_29_fuel_leak_signals_preprocessed.csv", "data/msn_37_fuel_leak_signals_preprocessed.csv", "data/msn_53_fuel_leak_signals_preprocessed.csv"]

datasets = []
for filename in filenames:
    datasets.append(pd.read_csv(filename, sep=";"))
    
print("Datasets loaded")

In [ ]:
datasets[1]

In [ ]:
for dataset in datasets:
    print(dataset.info(verbose=True))

In [ ]:
# Count NaN values
for dataset in datasets:
    # print percentage of NaN values if more than 0.8
    for column in dataset.columns:
        if dataset[column].isna().sum() > 0.7 * len(dataset[column]):
            print("NaN percentage in column " + column + ": " + str(dataset[column].isna().sum() / len(dataset[column])))
    

In [ ]:
# Correlation matrix for all datasets numercial values
#for dataset in datasets:
    #print(dataset.corr())


In [ ]:
# draw plots for all datasets for column LEAK_DETECTION_LEAK_FLOW
for i in range(len(datasets)):
    print(filenames[i], datasets[i].shape)

In [ ]:
# draw plots for all datasets for columns VALUE_FUEL_QTY_FT1;VALUE_FUEL_QTY_FT2;VALUE_FUEL_QTY_FT3;VALUE_FUEL_QTY_FT4;VALUE_FUEL_QTY_LXT;VALUE_FUEL_QTY_RXT
#for dataset in datasets:
 #   dataset.plot.scatter(x='UTC_TIME', y='VALUE_FUEL_QTY_FT1')
    #dataset.plot.scatter(x='UTC_TIME', y='VALUE_FUEL_QTY_FT2')
    #dataset.plot.scatter(x='UTC_TIME', y='VALUE_FUEL_QTY_FT3')
    #dataset.plot.scatter(x='UTC_TIME', y='VALUE_FUEL_QTY_FT4')
    #dataset.plot.scatter(x='UTC_TIME', y='VALUE_FUEL_QTY_LXT')
    #dataset.plot.scatter(x='UTC_TIME', y='VALUE_FUEL_QTY_RXT')
    

In [ ]:
#datasets[0].plot.scatter(x='UTC_TIME', y='VALUE_FUEL_QTY_FT1')

In [ ]:
# convert UTC_TIME to datetime
for dataset in datasets:
    dataset['UTC_TIME'] = pd.to_datetime(dataset['UTC_TIME'], format='%Y-%m-%d %H:%M:%S.%f')


In [ ]:
for dataset in datasets:
    #introduce column date
    dataset['DATE'] = dataset['UTC_TIME'].dt.date
    dataset["MONTH"] = dataset['UTC_TIME'].dt.month
    dataset["DAY"] = dataset['UTC_TIME'].dt.day
    dataset["HOUR"] = dataset['UTC_TIME'].dt.hour
    dataset["MINUTE"] = dataset['UTC_TIME'].dt.minute
    dataset["SECOND"] = dataset['UTC_TIME'].dt.second
    
# number of seconds since epoch 
import time 


In [ ]:
# plot for dataset 0 for column LEAK_DETECTION_LEAK_FLOW
datasets[0].plot.scatter(x='UTC_TIME', y='LEAK_DETECTION_LEAK_FLOW')

In [ ]:
import plotly.subplots as sp
import plotly.graph_objs as go

def plot_datasets(dataset):
    # List of columns to plot for VALUE_FUEL_QTY_CT;VALUE_FUEL_QTY_FT1;VALUE_FUEL_QTY_FT2;VALUE_FUEL_QTY_FT3;VALUE_FUEL_QTY_FT4;VALUE_FUEL_QTY_LXT;VALUE_FUEL_QTY_RXT
    fuel_qty_cols = ['VALUE_FUEL_QTY_CT', 'VALUE_FUEL_QTY_LXT', 'VALUE_FUEL_QTY_RXT', 'VALUE_FUEL_QTY_FT1', 'VALUE_FUEL_QTY_FT2', 'VALUE_FUEL_QTY_FT3', 'VALUE_FUEL_QTY_FT4']

    # Loop over every unique date
    for date in dataset['DATE'].unique():
        for flight in dataset['MSN'].unique():
            # Create a subplot
            fig = sp.make_subplots(rows=3, cols=3)
            
            # size of fig 
            fig.update_layout(height=1400, width=1400)

            # Loop over each column
            for i, col in enumerate(fuel_qty_cols):
                # save the dataset for the current date and flight
                dataset_tmp = dataset[(dataset['DATE'] == date) & (dataset['MSN'] == flight)]
                # Add scatter plot to subplot
                fig.add_trace(
                    go.Scatter(x=dataset_tmp['UTC_TIME'], 
                            y=dataset_tmp[col], 
                            mode='markers',
                            name=col),
                    row=i//3 + 1, 
                    col=i%3 + 1
                )

                # Update xaxis and yaxis titles
                fig.update_xaxes(title_text='UTC_TIME', row=i//3 + 1, col=i%3 + 1)
                fig.update_yaxes(title_text=col + " " + str(date), row=i//3 + 1, col=i%3 + 1)

            # Show the plot
            #fig.show()
            
            # save the plot as png
            fig.write_image("plots2/" + str(date) +"-" + str(flight) + ".png")
     
            
#plot_datasets(datasets[1])

for i in range(len(datasets)):
    if i > 1:
        #plot_datasets(datasets[i])

In [ ]:
# introduce new columns for each dataset [CT_DIFF, LXT_DIFF, RXT_DIFF, FT1_DIFF, FT2_DIFF, FT3_DIFF, FT4_DIFF]
# value represents the difference between the current and the previous value
for dataset in datasets:
    dataset['CT_DIFF'] = dataset['VALUE_FUEL_QTY_CT'].diff()
    dataset['LXT_DIFF'] = dataset['VALUE_FUEL_QTY_LXT'].diff()
    dataset['RXT_DIFF'] = dataset['VALUE_FUEL_QTY_RXT'].diff()
    dataset['FT1_DIFF'] = dataset['VALUE_FUEL_QTY_FT1'].diff()
    dataset['FT2_DIFF'] = dataset['VALUE_FUEL_QTY_FT2'].diff()
    dataset['FT3_DIFF'] = dataset['VALUE_FUEL_QTY_FT3'].diff()
    dataset['FT4_DIFF'] = dataset['VALUE_FUEL_QTY_FT4'].diff()  

In [ ]:
datasets[0].head()

In [ ]:
def plot_diff(): 
    #plot diff columns for dataset 0 for each unique date
    for date in datasets[0]['DATE'].unique():
        for flight in datasets[0]['MSN'].unique():
            # create subplot 3 columns and len(datasets[0].unique) rows
            fig = sp.make_subplots(rows=3, cols=3)
            
            # size of fig
            fig.update_layout(height=1000, width=1000)
            
            diff_cols = ['CT_DIFF', 'LXT_DIFF', 'RXT_DIFF', 'FT1_DIFF', 'FT2_DIFF', 'FT3_DIFF', 'FT4_DIFF']
            
            # loop over the diff columns
            for i, col in enumerate(diff_cols):
                # save the dataset for the current date and flight
                dataset_tmp = datasets[0][(datasets[0]['DATE'] == date) & (datasets[0]['MSN'] == flight)]
                # add scatter plot to subplot
                fig.add_trace(
                    go.Scatter(x=dataset_tmp['UTC_TIME'], 
                            y=dataset_tmp[col], 
                            mode='markers',
                            name=col),
                    row=i//3 + 1, 
                    col=i%3 + 1
                )

                # update xaxis and yaxis titles
                fig.update_xaxes(title_text='UTC_TIME', row=i//3 + 1, col=i%3 + 1)
                fig.update_yaxes(title_text=col + " " + str(date), row=i//3 + 1, col=i%3 + 1)
                
            # save fig as png 
            fig.write_image("plots2/diff_plots/" + str(date) + "-" + str(flight) + "_diff.png")

In [ ]:
# ideas
# 1. use lower bound and upper bound to detect outliers
# 2. use the bounds to plot the data and see if there are any outliers
# 3. deep learning auto detection anomaly detection

In [ ]:
# use deep learning to detect anomalies
# https://www.analyticsvidhya.com/blog/2019/01/introduction-time-series-classification/


In [ ]:
#!pip install pandas_profiling

In [ ]:
import pandas_profiling
prof = ProfileReport(datasets[0]) 
prof

In [ ]:
# function to generate synthetic data for each dataset including a simulated fuel leak
# using 0,5l, 1L and 5L as the leak size per minute

def generate_synthetic_data(dataset, leak_size):
    # create a copy of the dataset
    synthetic_dataset = dataset.copy()
    
    synthetic_dataset['VALUE_FUEL_QTY_CT'] = synthetic_dataset['VALUE_FUEL_QTY_CT'] - (leak_size / 60)
    
    return synthetic_dataset

## Calculate the fuel on board and the Fuel Used

In [ ]:
fuel_used_cols = ['FUEL_USED_1', 'FUEL_USED_2', 'FUEL_USED_3', 'FUEL_USED_4']

# save the sum of the fuel used columns in a new column
datasets[0]['FUEL_USED_SUM'] = datasets[0][fuel_used_cols].sum(axis=1)

fuel_cols = ['VALUE_FUEL_QTY_CT', 'VALUE_FUEL_QTY_LXT', 'VALUE_FUEL_QTY_RXT', 'VALUE_FUEL_QTY_FT1', 'VALUE_FUEL_QTY_FT2', 'VALUE_FUEL_QTY_FT3', 'VALUE_FUEL_QTY_FT4']
 
datasets[0]['FUEL_COLS_SUM'] = datasets[0][fuel_cols].sum(axis=1)

#datasets[0]['FUEL_DIFF_SUM'] = datasets[0]['FUEL_DIFF_SUM'].cumsum()

# substract from each value in FUEL_COLS_SUM the first value bigger than 1 and get the absoulte value. Do this for each day
datasets[0]['FUEL_COLS_SUM'] = datasets[0].groupby(['DATE'])['FUEL_COLS_SUM'].transform(lambda x: x - x[x > 1].iloc[0]).abs()


In [ ]:
import datetime 

datasets[0][datasets[0]['DATE'] == datetime.date(2011, 3, 3)]

In [ ]:
import plotly.graph_objs as go

# plot the fuel_used columns for dataset 0 for 2010-10-26
fig = go.Figure()
 
dates_to_find = datasets[0]['DATE'].unique()[0:1]

for date in dates_to_find:
    # plot FUEL_USED_SUM and FUEL_DIFF_SUM
    fig.add_trace(go.Scatter(x=datasets[0][datasets[0]['DATE'] == date_to_find]['UTC_TIME'],
                                y=datasets[0][datasets[0]['DATE'] == date_to_find]['FUEL_USED_SUM'],
                                mode='markers',
                                name='FUEL_USED_SUM'))
    fig.add_trace(go.Scatter(x=datasets[0][datasets[0]['DATE'] == date_to_find]['UTC_TIME'],
                                y=datasets[0][datasets[0]['DATE'] == date_to_find]['FUEL_COLS_SUM'],
                                mode='markers',
                                name='FUEL_COLS_SUM'))
    fig.show()